In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from KLT import KLTTracker
from SIFT import siftTrack

In [6]:
demo1 = cv2.imread("demo0.png")
h,w,d = demo1.shape
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(demo1,None)
cam = cv2.VideoCapture(0)

while(1):
    ret, frame=cam.read()
    frame = cv2.flip(frame,1)
    
    # Create a mask image for only tracking the interested features
    mask = np.zeros_like(frame)
    
    checker,dst = siftTrack(demo1,kp1,des1,frame,h,w)
    
    if checker != -1:
        
    
    keypressed = cv2.waitKey(5)
    if keypressed == 27:
        break
    

NameError: name 'cam' is not defined